# IT Security - Sheet 5 "Network Security"

**Total achievable points: 20**

**Released: 19.12.2024**

**Submission Deadline: 09.01.2025 13:00**

---
Groupnumber: 17

Names and matriculation numbers of **ALL** team members: 

Neo Ahrens (456647)

Christian Bick (456513)

Yorck Heilmann (456599)

---

**Important Information**

The assignments have to be submitted by groups of 4 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. Do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

All assertions provided by us should be considered part of the exercise description. Passing the assertions does not automatically mean that you'll get full points for the corresponding task, but failing the assertions pretty much guarantees that you'll get no points. 

**Do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

## 0. Setup

You will have to use a tool to inspect packet captures, like [WireShark](https://www.wireshark.org/), to solve Task 2.

---

Last exercise, you caught the dangerous Freddy Fraudster and his team, just before they were about to rob the bank *WeStoreLoadsOfMoneyForCharity*. Your proactive approach to IT-security caught the attention of the banks management, and your "Cosplaying as a sysadmin"-disguise got busted. But the management is delighted that the bank already has an IT-security expert in their ranks, and promotes you to "Charity IT-Security Success Story Advisor". It takes a bit of back and forth until you manage to decipher that job title: Your new job is to help the bank customers, i.e., charity organisations, in IT-security endeavors. This service is provided by the bank to minimize the risk of bad people like Freddy Fraudster stealing money from charities.

## Task 1. Merry (IP-)Secmas (10 points)

Your first advisee is the charity *PresentsForEveryone*, that tries to counteract Santa's tendency to give more or larger presents to children of high-income parents. This is a big operation, and they run two datacenters: One on the north pole, and one on the south pole. Both datacenter hold a complete copy of the very important present-coordination database. This database is updated continously, and it is very important that it is constantly kept up-to-date on both datacenters.

![Presents for everyone datacenters](presents-for-everyone.png)

---

For this reason, *PresentsForEveryone* pays a lot of money to the *Grinch-ISP*, which garuantees that packets flagged by the charity make will be delivered quickly. For this to work, the datacenter gateways use the Terms-of-Service field in the IP header.  However, these flags recently have been altered, leading to system instability. It is suspected that the Grinch-ISP tries to interfere with the operations of the charity to ruin christmas. You propose using IPSec to cheaply prevent these attacks.

### Task 1.1 Wrapped packets (5 points)

##### a) (2 points)

**Name** the IPSec protocol (ESP or AH) and the IPSec mode (transport or tunnel) that potentially could protect against the interference by the ISP.
Would this actually protect against the interference? **Justify** your answer. 

Protocol: AH (Authentication Header)

Mode: Tunnel Mode

AH provides integrity protection for as much of the IP header as possible, including some non-mutable fields. While AH detects modifications, it does not strictly "prevent" them; it causes the receiver to drop the packet if the integrity check fail.

##### b) (3 points)

As christmas comes closer, the Grinch becomes more aggressive in his behaviour and actively disrupts the logistics network, which is cooridinated using the database. Therefore, you propose encrypting the connection between teh database replicas.

**Name** two ways the connection could between the database replicas could be encrypted using the protocols presented in Chapter 6 of the lecture. **For each approach, briefly describe** one advantage it has over the other approach.

ESP in Transport Mode:

Only the IP payload is encrypted, the original IP header remains visible.
It has lower overhead than Tunnel Mode because no additional IP header is added, which is efficient for host-to-host communication between the database replicas.

ESP in Tunnel Mode:

The entire original IP packet is encrypted and encapsulated within a new IP packet.
It provides Traffic Flow Confidentiality because the actual internal IP addresses of the database servers are hidden from the ISP. Only the gateway addresses are visible.

### Task 1.2 Double the packets, half the fun (5 points)

It seems like the Grinch really hates *PresentsForEveryone*. It has moved on from interfering with the charities IT-infrastructure, and now physically attacks the logistics network. An initial attempt to just steal packets was quickly thwarted, but the Grinch improved, adapted, and overcame by switching to a more subtle approach: It now hides near the logistic centers and photographs the shipping labels of packets coming into logistic centers. Turns out that having two packets with the same shipping label can confuse a logistic center a lot. It took hours to fix wrong routings, potentially jeopardizing the presents for hunderts of kids.
Empowered by this initial success, the Grinch starts re-printing a lot more shipping labels. Now, the whole logistic network slowly crawls to a halt. 

The *PresentsForEveryone* charity of course tries to detect duplicated shipping labels, but the huge number of presents (they truely target everyone) means that storing already-seen shipping label IDs is infeasible. The second attempt tries to use the sequential and strictly incrementing nature of the shipping label IDs: It simply stores the ID of the last processed shipping label, and rejects all lower shipping label IDs. However, the packets do not always arrive perfectly sequentially. The way packets are packed into shipping containers often changes the order packets are processed during unpacking. Thus, the second approach would reject a lot of perfectly legitimate packets.

At some point, someone at *PresentsForEveryone* remembers that you mentioned IPSec's replay protection. They connect the dots, and notice that replay protection is pretty much the same as protecting logistic centers against duplicates, and they therefore ask you to re-implement IPSec's replay protection to prevent the Grinche's duplicated shipping labels, and to hopefully still deliver the presents in time.

---

**Implement** IPSec's replay protection as presented on Slide 28 of Chapter 6. The function `filter_replay` should return `True` if a sequence number is **legitimate**, and `False` if it is illegitimate.

*Hint: You can directly update the sliding window in the `filter_replay` function, i.e., you need not separate replay protection and window updates. Also, think about the about potential edge cases not covered by the public tests. Make sure that your implementation remains efficient, even in potential edge cases.*

In [ ]:
class ReplayProtection:
    window_size: int
    W: int # same as window_size, may be useful if you really want to cling to the slides.
    
    window: list[bool]
    N: int

    def __init__(self, window_size: int) -> None:
        self.window_size = self.W = window_size
        self.window = [False for _ in range(window_size)]
        self.N = 0

    def filter_replay(self, sequence_number: int) -> bool:
        # YOUR CODE HERE
        if sequence_number > self.N:
            diff = sequence_number - self.N

            if diff >= self.W:
                self.window = [False for _ in range(self.W)]
            else:
                self.window = self.window[diff:] + [False for _ in range(diff)]
            self.N = sequence_number
            self.window[self.W - 1] = True
            return True

        lower_border = self.N - self.W + 1
        if sequence_number >= lower_border:
            index = sequence_number - lower_border
            if self.window[index]:
                return False
            else:
                self.window[index] = True
                return True

        return False

In [ ]:
### PLAYGROUND

replay_protection = ReplayProtection(5)

def receive_packet(sequence_number: int) -> bool:
    assert len(replay_protection.window) == replay_protection.window_size
    handle = replay_protection.filter_replay(sequence_number)
    assert len(replay_protection.window) == replay_protection.window_size
    if handle:
        print(f"🎁 Handling packet number {sequence_number}.")
        return True
    else:
        print(f"🫷 Packet {sequence_number} is tagged as replay.")
        return False
    

receive_packet(1)
receive_packet(2)
receive_packet(4)
receive_packet(3)

receive_packet(2)

receive_packet(8)
receive_packet(12)
receive_packet(4)
receive_packet(5)
receive_packet(9)


In [ ]:
replay_protection = ReplayProtection(5)

def receive_packet(sequence_number: int) -> bool:
    assert len(replay_protection.window) == replay_protection.window_size
    handle = replay_protection.filter_replay(sequence_number)
    assert len(replay_protection.window) == replay_protection.window_size
    if handle:
        print(f"🎁 Handling packet number {sequence_number}.")
        return True
    else:
        print(f"🫷 Packet {sequence_number} is tagged as replay.")
        return False
    

assert receive_packet(1)
assert receive_packet(2)
assert receive_packet(4)
assert receive_packet(3)

assert receive_packet(2) is False

assert receive_packet(8)
assert receive_packet(12)
assert receive_packet(4) is False
assert receive_packet(5) is False
assert receive_packet(9)
print("🎁🎄🎁 You successfully prevented the Grinch's attack, and presents now are delivered at a record pace.")

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 2. Toast to the Longevity of Sylvester (TLS) (7.5 points)

The great success of your advise for the *PresentsForEveryone* nets you a nice end-of-year bonus (✨ better preparation for the exam ✨), and you can now enjoy a nice festive season with your family. But just as you picked a nice book and are fully prepared to lay back for a few days before New Year's Eve, you get a call from your boss: *"Hey, um, listen, I hate to do this to you during the holidays, but I really need your help right now. We just got a distress call from the Toast to the Longevity of Sylvester (TLS) charity."*.

You remember that this charity was recently presented at an internal meeting. It is developing a new super bio-degradable type of fireworks that still has all the fancy light show effects, but is much quieter to not scare dogs and other animals. They also already have developed a launching system that scans the sky for birds and times the fireworks accordingly. Turns out that this system is also really good at preventing other potentially-dangerous situation, thus significantly reducing the number of injured people. All of their developments are open-sourced, to make it as economically-viable as possible for firework manufacturers to pick up on their research and bring it to production. They do all that with the goal of having all the niceties of the Sylvester tradition while reducing the downsides as much as possible. 

But recently, they have been under attack by the *Patent Pending Prowler*, a nefarious patent troll. He tries to steal all the recent developments made by the charity and patenting them before they are open-sourced. And it seems like he was successful at infiltrating the charity: The sysadmins have noticed a huge traffic spike, and all kind of data is uploaded to an unknown server..

### Task 2.1 PITM on TLS (1.5 points)

The telemetry of the charity's main firewall shows that all the anomalous traffic is encrypted using TLS. Hence, the sysadmins want to somehow be able to look into the TLS encrypted traffic.

![TLS network](tls-charity.png)

---

**Describe** how it may be possible for the charity to decrypt the TLS-encrypted traffic at the firewall. Also **provide one argument** why this is a bad idea.

*Hints: You can assume that the sys-admins are able to make arbitrary configuration changes on the charity-owned devices.*

The charity can perform a Man-in-the-Middle attack at the firewall.

The firewall generates a fake certificate for the destination, signed by a custom Certificate Authority (CA) controlled by the sysadmins.

Ingle point of failure if the firewall gets breached

### Task 2.2 TLS Forensics (6 points)

While you were able to convince the sysadmins to not introduce deep packet inspection, you were not exactly helpful either. The *Patent Pending Prowler* has successfully stolen all recent advancements made by the TLS's R&D team, and filed a whole fury of overly vague patents based on them. But there is a silver lining: These patents are not fully in effect yet, they are just pending. TLS can still challenge these patents by proving that the patent troll has stolen all the intellectual property from the charity. Eager to help, you offer to take a look at the packet captures created by the firewall.

---

Take a look at the `tls-charity.pcap` file using a tool like [WireShark](https://www.wireshark.org) to answer to answer the following questions. If you use WireShark or TShark, you can use the filter "tls.handshake" to look at TLS handshake messages only.

In [ ]:
# We will use this function to allow for some variation in your answers.
# This should ensure that 10.00.00.01 and 10.0.0.1 are recognized as the same IP address.
# It also ignored digits. All-digit parts will be replaced with zero.
def normalize_ip_addresses(x: str | list[str]):
    if isinstance(x, str):
        return ".".join([str(int('0' + ''.join(c for c in part if c.isdigit()))) for part in x.split(".")])
    return [normalize_ip_addresses(address) for address in x]

##### a) (1 point)

Which devices from the internal subnet (`10.0.0.0/24`) use DNS-over-TLS (DoT)? 

*Hint: You can assume that all devices use the DNS server `1.1.1.1`.*

In [ ]:
def return_ip_addresses_using_dot() -> list[str]:
    return [
        # Enter the IP-addresses as follows:
        # obviously, replace x with the last number of the DNS address.
        "10.0.0.2",
        "10.0.0.5",
        "10.0.0.6",
        "10.0.0.7",
        "10.0.0.8",
        "10.0.0.9",
        "10.0.0.11"
    ]

def just_ignore_this_function() -> list[str]:
    # This function is necessary for some technical reasons. You can ignore it.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### b) (1.5 points)

The charity hosts three services whose domain ends in `tls.exercise`. What are their IP-addresses?

In [ ]:
def return_services_and_their_ip_addresses() -> dict[str, str]:
    # Only change the IP-addresses in this dictionary.
    return {
        "www.tls.exercise": "10.0.0.2",
        "mail.tls.exercise": "10.0.0.6",
        "git.tls.exercise": "10.0.0.8"
    }

def just_ignore_this_function() -> dict[str, str]:
    # This function is necessary for some technical reasons. You can ignore it.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### c) (1 points)

What domain name did the Patent Pending Prawler use for his command-and-control operations? Which devices int the internal subnet (`10.0.0.0/24`) did he infect?

*Hint: You can assume that all devices that attempt to connect to the command-and-control server are infected.*

In [ ]:
def return_domain_name_of_c_and_c_server() -> str:
    # Example answer (obviously not the right one):
    # return "www.tls.example"
    # YOUR CODE HERE
    raise NotImplementedError()

def return_ip_addresses_of_infected_devices() -> list[str]:
    return [
        # Enter the IP-addresses as follows:
        # obviusly, replace x with the last number of the DNS address.
        "10.0.0.5",
        "10.0.0.8",
        "10.0.0.11",
    ]

def just_ignore_this_function() -> list[str]:
    # This function is necessary for some technical reasons. You can ignore it.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### d) (0.5 points)

Why did the device `10.0.0.5` fail to establish a secure connection to the command-and-control server?

Specify your answer by uncommenting the corresponding line below. You can ignore any `raise ...` commands.

In [ ]:
def why_did_the_connection_fail() -> str:
    # return "Because the server does not support any of the device's Diffie-Hellman groups"
    # return "Because the server was offline"
    return "Because the server does not support any of the device's ciphers"
    # return "Because the device does not trust the Prawler's certificate"
    # return "Because the firewall blocked the connection"
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### e) (1.5 point)

What services did the Patent Pending Prawler access using an infected device?

*Hint: You may need to speculate which connections were used to send commands to an infected device, and which connection were used to exfiltrate collected data.*

In [ ]:
def return_accessed_services() -> dict[str, bool]:
    # Change "False" to "True" for services the Patent Pending Prawler accessed.
    return {
        "www.tls.example": True,
        "mail.tls.example": True,
        "git.tls.example": True
    }

def just_ignore_this_function() -> dict[str, bool]:
    # This function is necessary for some technical reasons. You can ignore it.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### f) (0.5 points)

You may have noticed that some devices make requests to `itsec.rwth-aachen.de`, using the TLS version 1.2. You also may have noticed that you can look at the certificate used to establish these connection. On the other hand, you cannot look at the certificates of connection established using TLS 1.3.

**Briefly explain** why you cannot see the certificates of TLS 1.3 connections.

TLS 1.2 is just the previous cersion of TLS. Users with TLS 1.3 can therefore still establishe TLS 1.2 connections as it is implemeneted in 1.3

### Epilogue

With your insightful analysis, the legal team was able to build a compelling case against the Patent Pending Prawler. After some unexpected headwinds, mainly a smear campain arguing that "Corporate Espionage Must Pay Off", you still manage to convince the judge that the Patent Pending Prawler stole the IP of the newest patents from the charity.

The Patent Pending Prawler is now behind bars, and you can now hopefully enjoy New Year's Eve :)

![The Patent Pending Prawler behind bars.](patent-pending-prawler.png) Generated using AI *(noticeable by the lack of feet)*

## 3. Exam Example Tasks (2 points)

### Task 3.1 - IPSec (1 points)

> This task is awarded with one only if it has been seriously addressed, regardless of whether the answer is correct.
> The verbosity of the feedback can vary depending on how much time the tutors have.

Assume the following IP packet given:

```
+--------------------+---------+
| Original IP Header | Payload |
+--------------------+---------+
```

*Hint: You can copy the notation above and adapt it in the tasks.*

#### a)

**Draw/List** all contents of the packet (including all headers and trailers) if it would be sent with AH in tunnel mode. Additionally, **mark** which parts are encrypted and which parts are integrity protected.

```
+----------------+-----------+--------------------+---------+
| New IP Header  | AH Header | Original IP Header | Payload |
+----------------+-----------+--------------------+---------+
```

Everything is integrity protected and nothing is encrypted

#### b)

Now, the packet is protected with ESP in transport mode. **Mark/List** which parts are encrypted and which ones are integrity protected.

```
+--------------------+-----+---------+-------------+-----+
| Original IP Header | ESP | Payload | ESP Trailer | MAC |
+--------------------+-----+---------+-------------+-----+
```

Encryped: Payload, ESP-Trailer

Integrity-Prot: Payload, ESP-Trailer, ESP-Header

### Task 3.2 - TLS (1 points)

> This task is awarded with one only if it has been seriously addressed, regardless of whether the answer is correct.
> The verbosity of the feedback can vary depending on how much time the tutors have.

In the figure below, a TLS 1.3 handshake in a simplified form is illustrated.

![](./tls.png)

**Answer** the following questions. **Justify** your answer.

#### a)

Does the depicted TLS 1.3 handshake use key transport or key agreement? **Justify** your answer.

Key Agreement as the key gets calculated by both and not just shared.

#### b)

From which perspective is key freshness guaranteed? **Justify** your answer.

Both as they can just both take new random numbers for the calculation.

#### c)

What has to be added in which message(s) so that the server is authenticated through a certificate.

After Server.Hello: Certificate and Certificate.Veryfication

#### d)

What happens if an attacker would change the SupportedAlgos parameter in the first message?

He could force a weaker and already cracked algorithm to be used...allowing easy breaking.

## 5. Feedback (0.5 points)

You made it through another assignment! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write anything. But don't worry, we do not grade the content itself!

10/10